## 爬取排名前50的动漫

In [140]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

def get_bilibili_top_anime():
    # 设置 Selenium WebDriver
    driver = webdriver.Chrome() 
    driver.get("https://www.bilibili.com/v/popular/rank/bangumi")
    # 等待 JavaScript 渲染（显式等待,可能需要根据实际情况调整等待时间）
    driver.implicitly_wait(10)  # 显式等待
    # 获取页面源代码并解析
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    # url = "https://www.bilibili.com/v/popular/rank/bangumi"
    # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    # response = requests.get(url, headers=headers)
    # print(response.status_code)
    # soup = BeautifulSoup(response.content, "html.parser")
    anime_list = []
    # print(soup)
    for item in soup.find_all("li", class_="rank-item"):
        # print(item)
        # rank = item.find("div", class_="num").text
        title = item.find("a", class_="title").text
        anime_list.append(title)

    # 关闭浏览器
    driver.quit()
    return anime_list[:50]  # 获取前50部动漫


bilibili_anime = get_bilibili_top_anime()
df_bilibili = pd.DataFrame(bilibili_anime, columns=['Title'])
df_bilibili.to_csv("rank.csv",index=False,encoding='utf-8')
print(df_bilibili)



                    Title
0                     迷宫饭
1                    怪兽8号
2                   我独自升级
3                  葬送的芙莉莲
4                  香格里拉边境
5   物理魔法使-马修- 神觉者候选人选拔考试篇
6         剧场版 黑色四叶草 魔法帝之剑
7                老夫老妻重返青春
8                    无名记忆
9               夜晚的水母不会游泳
10           吹响吧！上低音号 第三季
11              声优广播的台前幕后
12                 狩龙人拉格纳
13                  间谍过家家
14               咒术回战 第二季
15          青之驱魔师 岛根启明结社篇
16              鸭乃桥论的禁忌推理
17              间谍过家家 第二季
18                  名侦探柯南
19                神渴望着游戏。
20                   咒术回战
21                  黑色四叶草
22        齐木楠雄的灾难（日播&精选版）
23                   蓝色监狱
24              物理魔法使-马修-
25              鬼灭之刃 刀匠村篇
26                  孤独摇滚！
27      关于我转生变成史莱姆这档事 第二季
28         宝可梦 旅途 第一季（中配）
29         JOJO的奇妙冒险 黄金之风
30       我被逐出队伍后过上慢生活 第二季
31          关于我转生变成史莱姆这档事
32                   鬼灭之刃
33              名侦探柯南（中配）
34                四月是你的谎言
35                   碧蓝之海
36                假面骑士歌查德
37          

In [162]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import chardet

# with open('rank.csv', 'rb') as f:
#     result = chardet.detect(f.read())

# # 使用检测到的编码读取文件
# df_bilibili = pd.read_csv('rank.csv', encoding=result['encoding'])
df_bilibili = pd.read_csv('rank.csv', encoding='gbk')

def get_bilibili_top_anime_comments_and_tags(driver, anime_titles):
    anime_details = []
    for title in anime_titles:
        comments=[]
        driver.get(f"https://search.bilibili.com/all?keyword={title}")
        driver.implicitly_wait(10)

        #获取标签
        html_label = driver.page_source
        soup_label = BeautifulSoup(html_label, "html.parser")
        tags_section = soup_label.find('div', class_='media-card-content-head-label')
        #标签可以在最外面获取
        tags = tags_section.find_all('span')[0].text
        tags=tags[:-2].rstrip()
        print(tags)
        
        #点击视频进入视频页面
        search_results = driver.find_elements(By.XPATH, '//div[@class="media-card-content-head-title"]/a[@class="text_ellipsis"]')
        for link in search_results:
            if title in link.text:
                link.click()
                #重新绑定页面，不然获取不到新的html
                #获取当前页面句柄
                normal_window = driver.current_window_handle
                #获取所有页面句柄
                all_Handles = driver.window_handles
                #如果新的pay_window句柄不是当前句柄，用switch_to_window方法切换
                for pay_window in all_Handles:
                    if pay_window != normal_window:
                        driver.switch_to.window(pay_window)
                break

        # time.sleep(1)
        driver.implicitly_wait(10)
        #进入番剧的详情页面
        search_results_new = driver.find_elements(By.XPATH, '//div[@class="mediainfo_mediaRight__SDOq4"]/a[@class="mediainfo_mediaTitle__Zyiqh" ]')
        for link in search_results_new:
            if title in link.text:  # 获取链接的文本内容进行匹配
                link.click()
                #重新绑定页面，不然获取不到新的html
                #获取当前页面句柄
                normal_window = driver.current_window_handle
                #获取所有页面句柄
                all_Handles = driver.window_handles
                #如果新的pay_window句柄不是当前句柄，用switch_to_window方法切换
                for pay_window in all_Handles:
                    if pay_window != normal_window:
                        driver.switch_to.window(pay_window)
                break
        
        # time.sleep(1)
        driver.implicitly_wait(10)
        #进入长评页面
        search_results_new_new = driver.find_elements(By.XPATH, '//ul[@class="clearfix"]/li')
        for link in search_results_new_new:
            if "长评" in link.text:
                link.click()
                # #重新绑定页面，不然获取不到新的html
                # #获取当前页面句柄
                # normal_window = driver.current_window_handle
                # #获取所有页面句柄
                # all_Handles = driver.window_handles
                # #如果新的pay_window句柄不是当前句柄，用switch_to_window方法切换
                # for pay_window in all_Handles:
                #     if pay_window != normal_window:
                #         driver.switch_to.window(pay_window)
                break

        time.sleep(1)
        driver.implicitly_wait(10)

        com = driver.find_elements(By.XPATH, '//div[@class="review-content"][position() <= 20]')
        for comment in com:
            comments.append(comment.text)
            # print(comment.text)
        
        # #获取评论
        # html_comment = driver.page_source
        # soup_comment = BeautifulSoup(html_comment, "html.parser")
        # print(soup_comment)
        # tags_section = soup_comment.find('div', class_='review-detail')
        # #标签可以在最外面获取
        # tags = tags_section.find_all('div',)[1].text
        # print(tags)

        
        
        

        # # # 再次获取页面源代码，因为页面可能已经变化
        # html = driver.page_source
        # soup = BeautifulSoup(html, "html.parser")
        # print(soup)
        
        # comments_section = soup.find('div', class_='comment-container')
        # print(comments_section)
        # comments = comments_section.find_all('span', class_='creply-content') if comments_section else []
        # print(comments)

        
        anime_details.append({
            'Title': title,
            'Comments': comments,
            'Tags': tags
        })
        # pd.DataFrame(anime_details).to_csv("动漫.csv")
        # 返回上一页以重新搜索下一个动漫
        driver.back()
        # driver.close()
        
        
        
    
    return anime_details

# # 先前定义的获取动漫标题的函数
# bilibili_anime = get_bilibili_top_anime()
# df_bilibili = pd.DataFrame(bilibili_anime, columns=['Title'])

# 使用 Selenium 获取评论和标签
driver = webdriver.Chrome()  # 或使用其他浏览器驱动
anime_details = get_bilibili_top_anime_comments_and_tags(driver, df_bilibili['Title'])

# 关闭浏览器
driver.quit()

# 将获取到的评论和标签转换为 DataFrame
df_comments_tags = pd.DataFrame(anime_details)
df_comments_tags.to_csv("动漫.csv")
print(df_comments_tags)

恋爱/奇幻/战斗
漫画改/美食/冒险/搞笑
漫画改/战斗/冒险/奇幻
漫画改/奇幻/治愈/冒险
漫画改/战斗/奇幻/科幻
漫画改/战斗/奇幻/冒险
漫画改/日常/搞笑
漫画改/热血/战斗/奇幻
漫画改/推理
漫画改/日常/治愈/搞笑
漫画改/热血/战斗/奇幻
漫画改/搞笑/校园/恋爱
漫画改/奇幻/日常/搞笑
漫画改/运动/热血
漫画改/音乐/日常
架空/小说改/奇幻/战斗
冒险/游戏改/奇幻
冒险/漫画改/战斗/智斗
漫画改/日常/搞笑/穿越
漫画改/奇幻/热血/战斗
催泪/少女/音乐
社团/漫画改/搞笑/运动


ElementClickInterceptedException: Message: element click intercepted: Element <a href="//www.bilibili.com/bangumi/media/md102312" class="mediainfo_mediaTitle__Zyiqh" target="_blank" rel="noreferrer" title="...">碧蓝之海</a> is not clickable at point (311, 352). Other element would receive the click: <p data-v-06996fd8="">...</p>
  (Session info: chrome=124.0.6367.91)
Stacktrace:
	GetHandleVerifier [0x00007FF656A51502+60802]
	(No symbol) [0x00007FF6569CAC02]
	(No symbol) [0x00007FF656887CE4]
	(No symbol) [0x00007FF6568DE877]
	(No symbol) [0x00007FF6568DC344]
	(No symbol) [0x00007FF6568D980A]
	(No symbol) [0x00007FF6568D89E2]
	(No symbol) [0x00007FF6568CAE8B]
	(No symbol) [0x00007FF6568FAB7A]
	(No symbol) [0x00007FF6568CA7C6]
	(No symbol) [0x00007FF6568FAD90]
	(No symbol) [0x00007FF65691A224]
	(No symbol) [0x00007FF6568FA923]
	(No symbol) [0x00007FF6568C8FEC]
	(No symbol) [0x00007FF6568C9C21]
	GetHandleVerifier [0x00007FF656D5411D+3217821]
	GetHandleVerifier [0x00007FF656D960B7+3488055]
	Get

In [164]:
# df_comments_tags.to_csv("动漫.csv")
print(anime_details[6])

{'Title': '间谍过家家', 'Comments': [], 'Tags': '漫画改/日常/搞笑'}
